In [3]:
from __future__ import print_function
import sys
sys.path.append("..")
from util.util import lr_scheduler
from datasets.reconstructed_ES_imagenet import ESImagenet2D_Dataset
#from datasets.es_imagenet import ESImagenet_Dataset

from LIAFnet.LIAFResNet import *

import torch.distributed as dist
import torch.nn as nn
import argparse, pickle, torch, time, os,sys
from importlib import import_module
from tensorboardX import SummaryWriter
from torch.nn.parallel import DistributedDataParallel as DDP

#from torchvision.models import resnet18

##################### Step1. Env Preparation #####################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

   
##################### Step2. load in dataset #####################
modules = import_module('LIAFnet.LIAFResNet_18')
config  = modules.Config()
workpath = os.path.abspath(os.getcwd())
num_epochs = config.num_epochs
batch_size = config.batch_size
timeWindows = config.timeWindows
config.cfgCnn = [1,64,7]


epoch = 0
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
training_iter = 0
start_epoch = 0
acc_record = list([])
loss_train_record = list([])
loss_test_record = list([])

batch_size = 4


test_dataset = ESImagenet2D_Dataset(mode='test',data_set_path='/data/ES-imagenet-0.18/')
#test_dataset = ESImagenet_Dataset(mode='test',data_set_path='../data/Validation_Set/')
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2,pin_memory=True)


##################### Step3. establish module #####################
#modules = import_module('LIAFnet.LIAFResNet_18')
#config  = modules.Config()

cnn = LIAFResNet(config)
print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in cnn.parameters())))
cnn = LIAFResNet(config)
cnn.to(device)
pretrain_path = '../pretrained_model/42-Copy1.pkl'
checkpoint = torch.load(pretrain_path, map_location=torch.device('cpu'))
cnn.load_state_dict(checkpoint)

Total number of paramerters in networks is 11690097  


<All keys matched successfully>

In [4]:
################step4. training and validation ################

def val(cnn,test_loader,test_dataset,batch_size,epoch):
    cnn.eval()
    correct = 0
    total = 0
    predicted = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            if ((batch_idx+1)<=len(test_dataset)//batch_size):
                try:
                    #print("inside")
                    targets=targets.view(batch_size)#tiny bug
                    outputs = cnn(inputs.to(device))#.type(LIAF.dtype))
                    _ , predicted = outputs.cpu().max(1)
                    total += float(targets.size(0))
                    correct += float(predicted.eq(targets).sum())
                    
                    if batch_idx%100 == 0:
                        acc = 100. * float(correct) / float(total)
                        print(acc, batch_idx,'/',len(test_dataset)/batch_size)
        
                except:
                    print('sth. wrong')
                    print('val_error:',batch_idx, end='')
                    print('taret_size:',targets.size())
               
                #print(batch_idx,'/',len(test_dataset)/batch_size)

        acc = 100. * float(correct) / float(total)
    return acc

print('start validation')
acc = val(cnn,test_loader,test_dataset,batch_size,epoch=-1)
print('acc:',acc)

start validation
0.0 0 / 12470.25
0.0 100 / 12470.25
0.0 200 / 12470.25
0.0 300 / 12470.25
0.0 400 / 12470.25
0.0998003992015968 500 / 12470.25
0.08319467554076539 600 / 12470.25
0.1783166904422254 700 / 12470.25
0.1560549313358302 800 / 12470.25
0.13873473917869034 900 / 12470.25
0.12487512487512488 1000 / 12470.25
0.11353315168029064 1100 / 12470.25
0.10407993338884262 1200 / 12470.25
0.09607993850883935 1300 / 12470.25
0.08922198429693076 1400 / 12470.25
0.0832778147901399 1500 / 12470.25
0.07807620237351655 1600 / 12470.25
0.08818342151675485 1700 / 12470.25
0.0832870627429206 1800 / 12470.25
0.0920568122041031 1900 / 12470.25
0.08745627186406797 2000 / 12470.25
0.08329366968110423 2100 / 12470.25
0.07950931394820536 2200 / 12470.25
0.0869187309865276 2300 / 12470.25
0.0937109537692628 2400 / 12470.25
0.0899640143942423 2500 / 12470.25
0.08650519031141868 2600 / 12470.25
0.08330248056275454 2700 / 12470.25
0.08032845412352731 2800 / 12470.25
0.07755946225439504 2900 / 12470.25
0.08